In [1]:
# Imports
import os
import sys
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('dark')

In [2]:
# Get the username from terminal
username = sys.argv[1]

os.environ['SPOTIPY_CLIENT_ID'] = '509f1c0827f94cd7af9c842beb26e7ef'
os.environ['SPOTIPY_CLIENT_SECRET'] = '83fe375561354e6485bde0e952dc5d70'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:8888/callback/'

cid = '509f1c0827f94cd7af9c842beb26e7ef' # Client ID
secret = '83fe375561354e6485bde0e952dc5d70'   # Client Secret ID
username = '21ugtw76ynask7ohvvz626ndq' # Spotify Username

scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri = 'http://localhost:8888/'
#http://localhost:8888/callback/

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [3]:
genres = ['house', 'techno', 'dnb', 'hardstyle', 'trap']

df = pd.read_csv(r'C:\Users\Chetan\Desktop\College\Python\EDM Sub-genre classifier\playlists.csv')
df.head()

,username,playlistid,genre,title,url
0,Élite,5sPzNVd9MbUwmWd2mA90qB,house,Boiler Room | Techno House,https://open.spotify.com/playlist/5sPzNVd9MbUw...
1,Beatport,6Pbk82RPOKWD7k6dNcqq9f,house,Beatport Best New Tech House,https://open.spotify.com/playlist/6Pbk82RPOKWD...
2,Mac Peterson,7AUqarp6W3pf0tVFrRvzZw,house,Dirtybird Players,https://open.spotify.com/playlist/7AUqarp6W3pf...
3,Alex Meyer,1NQK1mGamaJmmwNE4iv9PT,house,House/Dirtybird,https://open.spotify.com/playlist/1NQK1mGamaJm...
4,Dave Lloyd,31QIVbCj6M2KkVxMepKS7b,house,Dirtybird//Techhouse by Mack,https://open.spotify.com/playlist/31QIVbCj6M2K...


In [4]:
def get_features(username, playlist_id, genre):
    results = sp.user_playlist_tracks(username, playlist_id)
    songs = results['items']
    
    while results['next']:
        results = sp.next(results)
        songs.extend(results['items'])
    
    ids = []
        
    for i in range(len(songs)):
        ids.append(songs[i]['track']['id'])
    
    ids = list(filter(None, ids))
    
    features = []
    
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        
        for track in audio_features:
            features.append(track)

    features = list(filter(None, features))
    df = pd.DataFrame(features)
    df['genre'] = genre
    return df


In [5]:
def create_df(PlayLists):
    result = pd.DataFrame()
    
    # Extract songs from spotify API and append to dataframe
    for i in range(len(PlayLists)):
        df = get_features(PlayLists['username'][i], PlayLists['playlistid'][i], PlayLists['genre'][i])
        df['title'] = PlayLists['title'][i]
        result = pd.concat([result, df], ignore_index=True)
    
    return result

In [6]:
def analyze_playlist(creator, playlist_id, ):
    
    # Create empty dataframelimit=2000
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [7]:
# These Functions can read more than 100 songs a playlist

def show_tracks(results, uriArray):
    for i, item in enumerate(results['items']):
        track = item['track']
        uriArray.append(track['id'])

def get_playlist_track_id(username, playlist_id):
    trackID = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks, trackID)
    while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks, trackID)
    return trackID

In [13]:
get_playlist_track_id('Mac Peterson', '7AUqarp6W3pf0tVFrRvzZw')

['2HQ1XA5iHCpnTJlVaqJ4iN',
 '3U3XlR0IBPw2kI3j4fTQDr',
 '4jIaYv7MiKIRydjMbUbYt2',
 '6KteefoBH4bLWYEpyBU52o',
 '7gVBHLfhOacNfcSya2aHYB',
 '6MN6eoEsqkGZHaLQSZDwk7',
 '1pAse8XvdfaYrleXSy6krt',
 '0QCl9rZxNa6wJDAcLawmy9',
 '6AWQoBfrxHaYA7oiq9Y63c',
 '4qetR2UUyBeUrJ9DbYrpVQ',
 '7bFOkwjxOcGaw6ZULE58Mh',
 '7ruylcpwEbeCl3HfB9SSyM',
 '4skEZYlTvkvTjONutvucWn',
 '1LQo4UFFuCISnMRaWbO9XO',
 '6auJhEHRn0kSNTFArpPRre',
 '5na0puVyrNoG1gu5gHElVv',
 '7yNuqaldaVOt1rTEShibtY',
 '2zYJXOC9Lh48cx31fGxuIf',
 '36EWuu7RLyidvdnVjuzgHI',
 '3Oj3j6k1fhfitFDs8FqNjI',
 '5OBigPOQEHjeESUP04VlSo',
 '2MI4HFRdd6gd9zc87YRjQk',
 '3u50rVvgy1hw1DOxPZOxxD',
 '0McK25ZTzzAdKCvYGiu6mM',
 '153kvYGgGxOxHNZFQCRw5q',
 '3fDgxdcf0jeJgGiZJGUmyK',
 '3lR1NFx6KY2LJ6hhGxDXl5',
 '3yPWyvPhyE1LiUGhIwdltx',
 '4gXn2AkFu1IHgZZLRAA49i',
 '1AuckzHgRyOhpfPqJZ103d',
 '3eIM5RzxNdAcHCardZGjvy',
 '6ppDgnUcK2QjIQHYy9dTq8',
 '1Es0X7EdjNW3QvK4WRQrEl',
 '2NxKraET9oqCtrdhleVY88',
 '35VEwFXgVPcFzbgBjOElDN',
 '7vtj0Lofn4XQgJPNtDarit',
 '60E3Me06pGE8eYy4RBfe1l',
 